## PbSO4 tutorial: indexing, spacegroup determination & structure solution
In this notebook, you can:
* Load the powder diffraction data and create the PowderPattern object, for two diffrection patterns (X-ray and neutron)
* Find the diffraction peaks and index them (determine the unit cell)
* Perform a profile fit to optimise the background and reflection profiles
* Determine the spacegroup
* Add a molecule to describe the contents of the Crystal structure
* Solve the Crystal structure using a Monte-Carlo/Parallel tempering algorithm, using both X-ray and neutron diffraction patterns
* Save the best result to a CIF file and to Fox .xmlgz format

Notes:
* This is a simple case, which illustrates the possibility of joint X-ray/neutron optimisation.
* It is important to follow the steps relatively linearly and avoid going back to previous cells until you know better. For example to avoid adding multiple times Scatterer/Molecule objects in the crystal structure, or multiple crystalline phases to the powder pattern with the same crystal, etc...

### Imports

In [ ]:
# 'widget' allows live update and works in both classical notebooks and jupyter-lab.
# Otherwise 'notebook', 'ipympl', 'inline' can be used
%matplotlib widget

import os
import pyobjcryst
import numpy as np
import matplotlib.pyplot as plt
from pyobjcryst.crystal import *
from pyobjcryst.scatteringpower import ScatteringPowerAtom
from pyobjcryst.atom import Atom
from pyobjcryst.polyhedron import MakeTetrahedron
from pyobjcryst.powderpattern import *
from pyobjcryst.radiation import RadiationType
from pyobjcryst.indexing import *
from pyobjcryst.molecule import *
from pyobjcryst.globaloptim import MonteCarlo
from pyobjcryst.io import xml_cryst_file_save_global

### Create powder pattern object, download data if necessary
We start with the X-ray data which will be used to determine the unit cell and spacegroup.

In [ ]:
px = PowderPattern()
if not os.path.exists("pbso4-x.dat"):
    os.system("curl -o pbso4-x.dat https://raw.githubusercontent.com/vincefn/objcryst/master/Fox/example/tutorial-pbso4/xray.dat")
px.ImportPowderPatternFullprof("pbso4-x.dat")
px.SetWavelength("Cu")  # Valid strings for X-ray tubes are "Cu", "CuA1",...
print(px.GetRadiation()) # Better check the string was understood
px.plot()

### Find peaks & index the reflections
In this case the peaks are automatically found without any parasitic phase.

And the unit cell is also indexed without any ambiguity. This uses the dichotomy in volume approach (Louër & Boultif).

... It is not always that easy !

In [ ]:
# Index
pl = px.FindPeaks(1.5, -1, 1000)
if len(pl) > 20:
    pl.resize(20)  # Only keep 20 peaks
for peak in pl:
    print(peak)

ex = quick_index(pl)

print("Solutions:")
for s in ex.GetSolutions():
    print(s)

### Create a crystal phase using the indexed unit cell

In [ ]:
uc = ex.GetSolutions()[0][0].DirectUnitCell()
c = pyobjcryst.crystal.Crystal(uc[0], uc[1], uc[2], uc[3], uc[4], uc[5], "P1")
pdiff = px.AddPowderPatternDiffraction(c)

# Plot with indexing in new figure
px.plot(diff=False,fig=None,hkl=True)

### Fit the profile and background
We use a maximum sin(theta)/lambda because we don't really need high angle/high resolution data.

This will go faster and is more reliable for spacegroup indexing and structure solution.

In [ ]:
px.SetMaxSinThetaOvLambda(0.3)
px.quick_fit_profile(auto_background=True,plot=False, init_profile=True,verbose=True)
px.quick_fit_profile(plot=False, init_profile=False, asym=True, displ_transl=True, verbose=False)

# Plot in new figure
px.plot(diff=True, fig=None, hkl=True)
print("Fit result: Rw=%6.2f%% Chi2=%10.2f  GoF=%8.2f  LLK=%10.3f" %
      (px.rw * 100, px.chi2, px.chi2/px.GetNbPointUsed(), px.llk))

### Find the spacegroup
The SpaceGroupExplorer can be used to find the optimal spacegroup. 

What `RunAll()` does is try all spacegroups and settings which are compatible with the unit cell (in this case all monoclinic and triclinic), and perform a profile fit (Le Bail only, we don't refine profile parameters or background since these parameters should be OK).

From this several values are extracted for each spacegroup setting:
* **Rw** - the standard full-profile weighted R factor $R_{wp}$
* **GoF**: the chi2 (full profile  $\chi^2=\Sigma\frac{(obs-calc)^2}{\sigma^2}$) divided by the number of points used
* **nGoF**: this is the Goodness-of-Fit, but computed on integration intervals defined by P1 reflections, and then multipled by the number of reflections used divided by the number of reflections for the P1 spacegroup. This is more discriminating and allows to put forward spacegroups which yield a good fit with more extinctions.
* **reflections** is the number of reflections actually taken into account for this spacegroup up to the maximum sin(theta)/lambda
* **extinct446** gives the number of extinct reflections for 0<=H<=4 0<=K<=4 0<=L<=6 (which is used internally as a unique fingerprint for the extinctions)

Some C++ verbose output does not appear here but will be in the jupyter server log if you see it.

The results are sorting by ascending **nGOF**

Unfortunately in this case the correct spacegroup (Pcmn, or Pnma if the axis were exchanged) is only one among possible choices (P21cn has the same extinctions), so we'll select it but in a real case, the different possible spacegroups would need to be tested.

In [ ]:
#px.SetMaxSinThetaOvLambda(0.3)  # This can be used to change the number of used reflections
spgex = SpaceGroupExplorer(pdiff)

# NB:verbose C++ output does not appear in a notebook
spgex.RunAll(keep_best=True, update_display=False, fitprofile_p1=False)

for sol in spgex.GetScores():
    #if sol.nGoF > 4 * spgex.GetScores()[0].nGoF:
    if sol.GoF <= 2 * spgex.GetScores()[0].GoF:
        print(sol)

c.GetSpaceGroup().ChangeSpaceGroup("Pmcn")
print("Chosen spacegroup (smallest nGoF): ", c.GetSpaceGroup())

# Updated plot with optimal spacegroup
px.plot(diff=True, fig=None, hkl=True, reset=True)

### Add SO4 and Pb to the crystal structure
First create the atomic scattering powers, then 

In [ ]:
pb = ScatteringPowerAtom("Pb", "Pb", 1.0)
s = ScatteringPowerAtom("S", "S", 1.5)
o = ScatteringPowerAtom("O", "O", 2.0)

# When manually creating atomic scattering power, they must be added
# to the Crystal. This is done automatically when importing a Molecule.
c.AddScatteringPower(pb)
c.AddScatteringPower(s)
c.AddScatteringPower(o)

c.AddScatterer(Atom(0.25,0.25,0.25,"Pb", pb))
c.AddScatterer(MakeTetrahedron(c,"SO4",s,o,1.5))

# Let's see what is the resulting crystal contents
print(c)

### Create the neutron powder diffraction data and fit the profile
The same steps as for the X-ray data are performed.

However the peak width is larger so that must be fixed to start

In [ ]:
pn = PowderPattern()
if not os.path.exists("pbso4-n.dat"):
    os.system("curl -o pbso4-n.dat https://raw.githubusercontent.com/vincefn/objcryst/master/Fox/example/tutorial-pbso4/neutron.dat")
pn.ImportPowderPatternFullprof("pbso4-n.dat")
pn.GetRadiation().SetRadiationType(RadiationType.RAD_NEUTRON)
pn.SetWavelength(1.909)
pn.SetMaxSinThetaOvLambda(0.3)
pdiffn = pn.AddPowderPatternDiffraction(c)

#
pdiffn.GetProfile().GetPar("W").SetValue(0.001)


pn.FitScaleFactorForIntegratedRw()

# Plot
pn.plot(diff=True, fig=None, hkl=True)

# Fit profile - we keep the unit cell fixed as it was already refined
pn.quick_fit_profile(auto_background=True,plot=True, init_profile=True, cell=False, verbose=True)
pn.quick_fit_profile(plot=False, init_profile=False, cell=False, asym=True, displ_transl=True,
                     backgd=False, verbose=False)

print("Fit result: Rw=%6.2f%% Chi2=%10.2f  GoF=%8.2f  LLK=%10.3f" %
      (pn.rw * 100, pn.chi2, pn.chi2/pn.GetNbPointUsed(), pn.llk))

### Create a MonteCarlo object and add objects (crystal, powder patterns) for optimisation

In [ ]:
mc = MonteCarlo()
mc.AddRefinableObj(c)
mc.AddRefinableObj(px)
mc.AddRefinableObj(pn)

### Disable profile fitting before Monte-Carlo
..or else the crystal structure will not be optimised

Note that the following display will be live-updated during the optimisation done below (the last plot is always updated).

In [ ]:
pdiff.SetExtractionMode(False)
pdiffn.SetExtractionMode(False)

px.FitScaleFactorForRw()
pn.FitScaleFactorForRw()

pn.plot(fig=None,diff=False,hkl=True)
px.plot(fig=None,diff=False,hkl=True)

### Display the 3D crystal structure
*Note: this requires installing `ipywidgets` and `py3Dmol` (as of 2021/05 the conda-forge version is obsolete, so just install using pip). Otherwise You will just get a warning message*

This will be updated live during the optimisation, and also when using `RestoreParamSet()` to restore some specific solutions (and generally everytime the underlying Crystal's `UpdateDisplay()` function is called). Just scroll back to see what is being done in the widget.

The `display()` is only really necessary to make sure the widget appears in the notebook. In fact if `c.widget_3d()` is the *last* command in the notebook cell, the display is done automatically. See the ipywidgets documentation if you want to understand this in more details.

Note that bonds may disappear during optimisation, because they are automatically assigned by the javascript viewer, which is quite strict about allowed distances. In the final solution some bonds in the middle of the chain are often missing, though you can see the atoms are reasonably close. But rest assured that any bond defined in the object still exists as defined in pyobjcryst !

In [ ]:
display(c.widget_3d())

### Run multiple optimisations
We also enable the automatic least squares every 150k trials, which allows a better convergence

We perform 3 runs, each of 1 million trials using parallel tempering, with default parameters (which should be adequate for all problems). Normally for this structure it would be better to use 2 millions trials so that the correct solution is found during almost every run.

Each run starts from a randomised configuration.

In [ ]:
mc.GetOption("Automatic Least Squares Refinement").SetChoice(2)
print("LSQ option: ", mc.GetOption("Automatic Least Squares Refinement").GetChoiceName(2))

# 3D structure view which will be live-updated with the best
# configuration of the current run
display(c.widget_3d())

# Small widget to see the progress of the optimisation, with the current run
# best log-likelihood, the run number and remaining number of trials.
display(mc.widget())

# The powder pattern plot a few cells above should also be updated for each run best solution
mc.MultiRunOptimize(nb_run=3, nb_step=1e5)
print("Final LLK: %.2f" % mc.GetLogLikelihood())

### List solutions
All solutions are stored in a "Parameter Set" which can be restored (assuming that the objects - crystal structure and powder pattern are not altered e.g. by changing the list of atoms, the profile, or the fixed parameters etc...).

This will only record changes of parameters such as atom coordinates, but will not record other changes such as a different spacegroup, or a change of the Scatterers (number of atoms or molecules) inside a Crystal. It can only be used to browse results obtained at the end of `MultiRunOptimize()`.

At the end of the optimisation the best solution is automatically restored.

In [ ]:
for i in range(mc.GetNbParamSet()):
    idx = mc.GetParamSetIndex(i)
    cost = mc.GetParamSetCost(i)
    name = mc.GetFullRefinableObj().GetParamSetName(idx)
    print("%3d: LLK=%10.2f, name=%s"%(idx, cost, name))

### Restore a chosen solution (set of parameters)
Restoring a solution will also update the 3D crystal view above.

In [ ]:
pn.plot(fig=None, diff=True)
px.plot(fig=None, diff=True)
mc.RestoreParamSet(3, update_display=True)

### Save results to CIF and Fox (.xmlgz) formats

In [ ]:
# Save result so it can be opened by Fox
xml_cryst_file_save_global('result-pbso4.xmlgz')
# Also export to the CIF format
c.CIFOutput("result-pbso4.cif")